### Data annotation:
- https://tecoholic.github.io/ner-annotator/

### Setup

In [3]:
from IPython.display import clear_output

In [4]:
!pip install -U spacy -q
!python -m spacy info
clear_output()

### Import Libraries

In [5]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("id")
db = DocBin()

### Load Dataset

In [8]:
import json

f = open("annotations.json")
TRAIN_DATA = json.load(f)

In [9]:
TRAIN_DATA

{'classes': ['PERUSAHAAN', 'TEKNOLOGI', 'NEGARA', 'SEKTOR'],
 'annotations': [['Istilah metaverse sudah tidak asing lagi di telinga kita. Baru-baru ini, Microsoft mengakuisisi game developer Activision Blizzard. Mereka mengatakan bahwa akuisisi ini akan membantu membangun dunia metaverse mereka. Pendiri Facebook, Mark Zuckerberg, mengganti nama perusahaan platform media sosialnya dari Facebook menjadi Meta. Google juga telah mengerjakan teknologi terkait metaverse selama bertahun-tahun. Tetapi dengan semua inisiatif terkait metaverse dari raksasa-raksasa teknologi ini, apa sebenarnya arti dari metaverse? Lebih pentingnya lagi, apakah metaverse sudah ada di Indonesia?',
   {'entities': [[8, 17, 'TEKNOLOGI'],
     [73, 82, 'PERUSAHAAN'],
     [111, 131, 'PERUSAHAAN'],
     [199, 208, 'TEKNOLOGI'],
     [225, 233, 'PERUSAHAAN'],
     [308, 316, 'PERUSAHAAN'],
     [325, 330, 'PERUSAHAAN'],
     [331, 337, 'PERUSAHAAN'],
     [450, 459, 'TEKNOLOGI'],
     [585, 594, 'NEGARA']]}],
  ['Terla

### Data Processing

In [13]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot["entities"]:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
    else:
      ents.append(span)
  doc.ents = ents
  db.add(doc)

db.to_disk("model.spacy")

100%|██████████| 5/5 [00:00<00:00, 842.60it/s]


In [16]:
!python -m spacy init config config.cfg --lang id --pipeline ner --optimize efficiency

2022-08-26 15:55:32.669404: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: id
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


### Training the NER model

In [17]:
!python -m spacy train config.cfg --output ./ --paths.train ./model.spacy --paths.dev ./model.spacy

2022-08-26 15:55:42.246530: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-08-26 15:55:46,378] [INFO] Set up nlp object from config
[2022-08-26 15:55:46,390] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-08-26 15:55:46,395] [INFO] Created vocabulary
[2022-08-26 15:55:46,396] [INFO] Finished initializing nlp object
[2022-08-26 15:55:46,586] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     35.44    0.00    0.00    0.00    0.00
 24     200        406

### NER Model Testing

In [18]:
ner = spacy.load('model-best')

In [34]:
doc = ner('''
Istilah metaverse sudah tidak asing lagi di telinga kita. Baru-baru ini, Microsoft mengakuisisi game developer Activision Blizzard. Mereka mengatakan bahwa akuisisi ini akan membantu membangun dunia metaverse mereka. Pendiri Facebook, Mark Zuckerberg, mengganti nama perusahaan platform media sosialnya dari Facebook menjadi Meta. Google juga telah mengerjakan teknologi terkait metaverse selama bertahun-tahun. Tetapi dengan semua inisiatif terkait metaverse dari raksasa-raksasa teknologi ini, apa sebenarnya arti dari metaverse? Lebih pentingnya lagi, apakah metaverse sudah ada di Indonesia?

Metaverse akan terus berkembang, dan ini membuka berbagai peluang untuk industri telekomunikasi. Perusahaan telekomunikasi digital merupakan kunci untuk membangun platform metaverse melalui ekosistem kemitraan yang luas, analitik canggih melalui AI, dan human interface hardware. Selain itu, privasi dan keamanan siber harus dijaga oleh seluruh pemain di industri telekomunikasi. Yang paling penting, industri telekomunikasi merupakan penghubung utama dalam memastikan metaverse dapat diakses seluruh masyarakat Indonesia.

Di sisi lain, potensi adaptasi metaverse yang sangat besar di Indonesia adalah dalam sektor pariwisata. Metaverse dapat digunakan untuk memperkenalkan dan mempertunjukkan kekayaan budaya, makanan, dan destinasi wisata di seluruh Indonesia. Penggunaan metaverse pun akan mendorong UMKM lokal, dan ini krusial karena UMKM di Indonesia merupakan pendorong utama dari pertumbuhan ekonomi negara.
''')

In [35]:
colors = {"perusahaan": "#85C1E9", "teknologi":"#ff6961", "sektor": "#845EC2"}
options = {"ents": ["perusahaan", "teknologi", "sektor"], "colors": colors}

spacy.displacy.render(doc, style="ent", jupyter=True, options=options)